Run these command to download the dataset:
```sh
wget http://files.fast.ai/data/dogscats.zip
unzip dogscats.zip
```

In [1]:
!ls dogscats/

models	sample	test1  train  valid


In [2]:
import tqdm
import torch
import torch.optim as opt
import torch.nn as nn
from torch.utils import data as td
import torchvision as tv
import torchvision.transforms as tr
from torchvision.models import resnet34

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
train_trans = tr.Compose([
    tr.RandomResizedCrop(224),
    tr.RandomHorizontalFlip(),
    tr.ToTensor(), 
    tr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
train_iter = tv.datasets.ImageFolder('dogscats/train', transform=train_trans)
train_loader = td.DataLoader(train_iter, batch_size=16, shuffle=True, drop_last=True)

In [6]:
len(train_loader)

1437

In [7]:
valid_num_imgs = !find dogscats/valid/ -type f  | wc -l
valid_num_imgs = int(valid_num_imgs[0])
valid_num_imgs

2000

In [8]:
valid_trans = tr.Compose([
    tr.CenterCrop(224), 
    tr.ToTensor(), 
    tr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
valid_iter = tv.datasets.ImageFolder('dogscats/valid', transform=valid_trans)
valid_loader = td.DataLoader(valid_iter, batch_size=16, shuffle=False, drop_last=True)

In [10]:
net = resnet34(pretrained=True)
layers = list(net.children())
last_layer = layers[-1]
last_layer.in_features, last_layer.out_features

(512, 1000)

In [11]:
for p in net.parameters():
    p.requires_grad = False

In [12]:
ult = nn.Sequential(
    nn.BatchNorm1d(num_features=last_layer.in_features),
    nn.Dropout(p=.5),
    nn.Linear(in_features=last_layer.in_features, out_features=2)
)
net.fc = ult
# net.fc = nn.Linear(in_features=last_layer.in_features, out_features=2)

In [13]:
net = net.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = opt.Adam(net.fc.parameters(), lr=0.01, weight_decay=1e-4)

In [15]:
def accuracy():
    net.eval()
    total = 0
    correct= 0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicts = torch.max(outputs, 1)
            total += labels.size()[0]
            correct += (predicts == labels).sum().cpu().item()
    return correct/total

In [16]:
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    net.train()
    for data in tqdm.tqdm_notebook(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(running_loss, accuracy())


419.43814798630774 0.966


Left to do:
1. ~~Freeze lower resnet layers~~
1. ~~Add validation set and accuracy~~
1. Why accuracy is slow? Cache validation set and move to device
1. ~~Improve accuracy!~~ -> Moved to resnet101, resnet 34 was too small (?)
1. Smaller batches?
1. Why is resnet34 from torchvision different from the one in fast.ai?
1. Print loss and accuracy in tdqm
1. Better transformations, take from fast.ai
1. LR scheduling and optimization -> Currently second epoch losses accuracy
1. Different optimizers and hyper-parameters